In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Set the path to the data
data_dir = '/content/drive/My Drive/SRI'
csv_file = '/content/drive/My Drive/SRI/english.csv'


In [ ]:
# Load the CSV file
data = pd.read_csv(csv_file)

data

,image,label
0,Img/img001-001.png,0
1,Img/img001-002.png,0
2,Img/img001-003.png,0
3,Img/img001-004.png,0
4,Img/img001-005.png,0
...,...,...
3405,Img/img062-051.png,z
3406,Img/img062-052.png,z
3407,Img/img062-053.png,z
3408,Img/img062-054.png,z


In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data['image'], data['label'], test_size=0.3, random_state=42)


In [ ]:


# Define the data augmentation parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
# Create the training data generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data.loc[X_train.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)


Found 2387 validated image filenames belonging to 62 classes.


In [ ]:
# Create the validation data generator (without augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=data.loc[X_val.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)

Found 1023 validated image filenames belonging to 62 classes.


In [ ]:
# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(400, 300, 3))


29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(62, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=output)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20
75/75 [==============================] - 1540s 20s/step - loss: 3.5947 - accuracy: 0.1680 - val_loss: 2.9176 - val_accuracy: 0.3109
Epoch 2/20
75/75 [==============================] - 121s 2s/step - loss: 2.5630 - accuracy: 0.4646 - val_loss: 2.1675 - val_accuracy: 0.5611
Epoch 3/20
75/75 [==============================] - 121s 2s/step - loss: 2.0206 - accuracy: 0.5995 - val_loss: 1.7536 - val_accuracy: 0.6608
Epoch 4/20
75/75 [==============================] - 119s 2s/step - loss: 1.6813 - accuracy: 0.6808 - val_loss: 1.4909 - val_accuracy: 0.6686
Epoch 5/20
75/75 [==============================] - 119s 2s/step - loss: 1.4609 - accuracy: 0.7097 - val_loss: 1.3226 - val_accuracy: 0.7146
Epoch 6/20
75/75 [==============================] - 120s 2s/step - loss: 1.2986 - accuracy: 0.7302 - val_loss: 1.2122 - val_accuracy: 0.7146
Epoch 7/20
75/75 [==============================] - 120s 2s/step - loss: 1.1829 - accuracy: 0.7486 - val_loss: 1.1091 - val_accuracy: 0.7312
Epoch 8/20
